In [1]:
from IPython.display import Code

# Example: Per-Layer Model Benchmarking

When investigating why a given model performs not as expected or implementing optimiations for specific types of layers, it is often useful to consider the runtime individual layers instead of the end-to-end execution time.

MLonMCU currently supports to approaches for per-layer benchmarking:
1. Using the `split_layers` feature of the `tflite` frontend
2. Using the profiling feature provided by the `tvm` and `microtvm` (WIP) platform

Both use-cases are explained briefly in the rest of this notebook.

## 1. Splitting TFLite Models into individual layers

### Supported components

**Models:** Any (`resnet` used below)

**Frontends:** `tflite` only

**Frameworks/Backends:** Any (`tvmaotplus` used below)

**Platforms/Targets:** Any (`etiss` used below)

**Features:** The `split_layers` feature of the `tflite` frontend needs to be enabled

### Prerequisites

If not done already, setup a virtual python environment and install the required packages into it. (See `requirements.txt`)

In [2]:
Code(filename="requirements.txt")

-r ../../requirements.txt
attrs
cloudpickle
decorator
numpy
psutil
scipy
synr
tornado
typing_extensions
tflite

Set up MLonmCU as usual, i.e. initialize an environment and install all required dependencies. Feel free to use the following minimal `environment.yml.j2` template:

In [3]:
Code(filename="environment.yml.j2")

---
# The MLONMCU_HOME is filled in automatically when creating the environment
home: "{{ home_dir }}"
logging:
  level: INFO
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 5
# Default locations for certain directoriescan be changed here
# Non-absolute paths will always be threated relative to the MLONMCU_HOME
paths:
  # Where the dependencies are downloaded and installed
  deps: deps
  # If logging to file is used keep logs in this directory
  logs: logs
  # Location where reports and artifacts are written to
  results: results
  # Directory where custom extensions can be integrated
  plugins: plugins
  # Directory for intermediate build products, should be located on a large enough drive
  temp: temp
  # A collection of models which will be used to look for models
  # The paths will be checked in the order defined here stopping at the first match
  # Non-existant paths will be skipped without throwing an error
  models:
    - "{{ home_dir }}/models"
# Here default clone_urls
repos:
  tvm:
    url: "https://github.com/apache/tvm.git"
    ref: a9fcac1a47f4b57c1d9d828c7ed1d77452b02cc5
    options:
      recursive: true
  etiss:
    url: "https://github.com/tum-ei-eda/etiss.git"
    ref: 40c0b920cf4072adf189211ca44291534355dcfd
  mlif:
    url: "https://github.com/tum-ei-eda/mlonmcu-sw.git"
    ref: c7b0e7364c232666ebd5a17476181e3ff9a71d70
  tflite_pack:
    url: "https://github.com/tum-ei-eda/tflite-pack.git"
    ref: 2d31cafc941d3889b042af9c0a72b5f768ac4606
frameworks:
  default: tvm
  tvm:
    enabled: true
    backends:
      default: tvmaotplus
      tvmaotplus:
        enabled: true
        features: {}
      tvmllvm:
        enabled: true
        features: {}
    features: {}
frontends:
  tflite:
    enabled: true
    features:
      split_layers: true
toolchains:
  gcc: true
platforms:
  mlif:
    enabled: true
    features:
      debug: true
  tvm:
    enabled: true
    features:
      tvm_profile: true
targets:
  default: host_x86
  etiss:
    enabled: true
    features:
      log_instrs: true
  host_x86:
    enabled: true
    features: []
  tvm_cpu:
    enabled: true
    features: []
postprocesses:
  use: []
vars:
  allow_extensions: false
  runs_per_stage: true
  riscv_gcc.dl_url: "https://syncandshare.lrz.de/dl/fiD9HnHYkb2V8kNbVQYv242m/rv32gc.tar.gz"
flags: {}

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

### Usage

The following examples demonstrate the `split_layers` feature recently added MLonMCU.

#### A) Command Line Interface

First define a simple benchmark of a single model/backend/target combination:

In [4]:
!mlonmcu flow run resnet --backend tvmaotplus --target etiss_pulpino

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-55]  Processing stage LOAD
INFO - [session-55]  Processing stage BUILD


INFO - [session-55]  Processing stage COMPILE


INFO - [session-55]  Processing stage RUN


INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-55] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework     Backend Platform         Target  Total Cycles  Total Instructions  Total CPI  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data  Validation Features                                             Config Postprocesses Comment
0       55    0  resnet   tflite       tvm  tvmaotplus     mlif  etiss_pulpino      81173677            81173677        1.0     225016     108224         163424     61448       144      2492              105732        True       []  {'resnet.output_shapes': {'Identity_int8': [1,...            []       -


Now lets enable the `split_layers` feature:

In [5]:
!mlonmcu flow run resnet --backend tvmaotplus --target etiss_pulpino -f split_layers

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-56]  Processing stage LOAD


INFO - [session-56]  Processing stage BUILD


INFO - [session-56]  Processing stage COMPILE


INFO - [session-56]  Processing stage RUN


INFO - All runs completed successfuly!


INFO - Postprocessing session report
INFO - [session-56] Done processing runs
INFO - Report:
    Session  Run   Model Frontend Framework     Backend Platform         Target      Sub  Total Cycles  Total Instructions  Total CPI  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data  Validation        Features                                             Config Postprocesses Comment
0        56    0  resnet   tflite       tvm  tvmaotplus     mlif  etiss_pulpino      NaN      81173677            81173677        1.0     225016     108224         163424     61448       144      2492              105732        True  [split_layers]  {'resnet.output_shapes': {'Identity_int8': [1,...            []       -
1        56    0  resnet   tflite       tvm  tvmaotplus     mlif  etiss_pulpino   layer0       3541354             3541354        1.0      56482      35152           4184     52154       144      2492               32660        True  [split_layers]  {'resnet.outp

The resulting report should contain the the original benchmark results (for the whole model) in the first row. The remaining 16 rows are for each of the layers found in the `resnet.tflite` model. The layer-index can be found in the 'Sub' column. The cycle count of these should roughly sum up to the total execution time measured in row one.

#### B) Python Scripting

Some imports

In [6]:
from tempfile import TemporaryDirectory
from pathlib import Path
import pandas as pd

from mlonmcu.context.context import MlonMcuContext
from mlonmcu.session.run import RunStage

Benchmark Configuration

In [7]:
FRONTEND = "tflite"
MODEL = "resnet"
BACKEND = "tvmaotplus"
PLATFORM = "mlif"
TARGET = "etiss_pulpino"
FEATURES = ["split_layers"]
CONFIG = {"filter_cols.keep": ["Sub", "Total Instructions", "Total ROM", "Total RAM"]}
POSTPROCESSES = ["filter_cols"]

Initialize and run a single benchmark

In [8]:
with MlonMcuContext() as context:
    with context.create_session() as session:
        run = session.create_run(config=CONFIG)
        run.add_features_by_name(FEATURES, context=context)
        run.add_frontend_by_name(FRONTEND, context=context)
        run.add_model_by_name(MODEL, context=context)
        run.add_backend_by_name(BACKEND, context=context)
        run.add_platform_by_name(PLATFORM, context=context)
        run.add_target_by_name(TARGET, context=context)
        run.add_postprocesses_by_name(POSTPROCESSES)
        session.process_runs(context=context)
        report = session.get_reports()
assert "Failing" not in report.df.columns
report.df

INFO - Loading environment cache from file


INFO - Successfully initialized cache


INFO - [session-57] Processing all stages


INFO - All runs completed successfuly!


INFO - Postprocessing session report


INFO - [session-57] Done processing runs


,Sub,Total Instructions,Total ROM,Total RAM
0,NaN,81173677,225016,108224
1,layer0,3541354,56482,35152
2,layer1,12854741,59894,105136
3,layer2,12889467,59892,105136
4,layer3,629213,54266,51768
5,layer4,6282362,64940,94800
6,layer5,17491971,73884,56112
7,layer6,1089453,56802,90704
8,layer7,316120,54304,27192
9,layer8,8802585,92958,49776


Stripping out all common data, we get this:

In [9]:
df = report.df
df.fillna("full", inplace=True)
df.set_index("Sub", inplace=True)
df

,Total Instructions,Total ROM,Total RAM
Sub,,,
full,81173677,225016,108224
layer0,3541354,56482,35152
layer1,12854741,59894,105136
layer2,12889467,59892,105136
layer3,629213,54266,51768
layer4,6282362,64940,94800
layer5,17491971,73884,56112
layer6,1089453,56802,90704
layer7,316120,54304,27192


## 2. Using ~~(Micro)~~TVMs profiling functionality

Instead of splitting the model layer wise before optimization, this will use the functionality of TVMs graph runtime to benchmark individual functions conatine din the model graph. These functions do not nessessarily map directly to a single layer in the original model operator fusing is automatically performed by TVMs compilation pipeline.

### Supported components

**Models:** Any (`resnet` used below)

**Frontends:** Any frontend supported by TVM (`tflite` used below)

**Frameworks/Backends:** TVM: `tvmllvm` ~~MicroTVM: `tvmrt`~~

**Platforms/Targets:** TVM: `tvm_cpu` ~~MicroTVM: Any~~

**Features:** The `tvm_profile` feature needs to be enabled

Let's only consider the `tvm_cpu` target here until this is supported officially by upstream TVM. Hence we are profiling on the host cpu here, not on a MCU or simulator. 

### Prerequisites

If not done already, setup a virtual python environment and install the required packages into it. (See `requirements.txt`)

In [10]:
Code(filename="requirements.txt")

-r ../../requirements.txt
attrs
cloudpickle
decorator
numpy
psutil
scipy
synr
tornado
typing_extensions
tflite

Set up MLonmCU as usual, i.e. initializa an environment and install all required dependencies. Feel free to use the following minimal `environment.yml.j2` template:

In [11]:
Code(filename="environment.yml.j2")

---
# The MLONMCU_HOME is filled in automatically when creating the environment
home: "{{ home_dir }}"
logging:
  level: INFO
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 5
# Default locations for certain directoriescan be changed here
# Non-absolute paths will always be threated relative to the MLONMCU_HOME
paths:
  # Where the dependencies are downloaded and installed
  deps: deps
  # If logging to file is used keep logs in this directory
  logs: logs
  # Location where reports and artifacts are written to
  results: results
  # Directory where custom extensions can be integrated
  plugins: plugins
  # Directory for intermediate build products, should be located on a large enough drive
  temp: temp
  # A collection of models which will be used to look for models
  # The paths will be checked in the order defined here stopping at the first match
  # Non-existant paths will be skipped without throwing an error
  models:
    - "{{ home_dir }}/models"
# Here default clone_urls
repos:
  tvm:
    url: "https://github.com/apache/tvm.git"
    ref: a9fcac1a47f4b57c1d9d828c7ed1d77452b02cc5
    options:
      recursive: true
  etiss:
    url: "https://github.com/tum-ei-eda/etiss.git"
    ref: 40c0b920cf4072adf189211ca44291534355dcfd
  mlif:
    url: "https://github.com/tum-ei-eda/mlonmcu-sw.git"
    ref: c7b0e7364c232666ebd5a17476181e3ff9a71d70
  tflite_pack:
    url: "https://github.com/tum-ei-eda/tflite-pack.git"
    ref: 2d31cafc941d3889b042af9c0a72b5f768ac4606
frameworks:
  default: tvm
  tvm:
    enabled: true
    backends:
      default: tvmaotplus
      tvmaotplus:
        enabled: true
        features: {}
      tvmllvm:
        enabled: true
        features: {}
    features: {}
frontends:
  tflite:
    enabled: true
    features:
      split_layers: true
toolchains:
  gcc: true
platforms:
  mlif:
    enabled: true
    features:
      debug: true
  tvm:
    enabled: true
    features:
      tvm_profile: true
targets:
  default: host_x86
  etiss:
    enabled: true
    features:
      log_instrs: true
  host_x86:
    enabled: true
    features: []
  tvm_cpu:
    enabled: true
    features: []
postprocesses:
  use: []
vars:
  allow_extensions: false
  runs_per_stage: true
  riscv_gcc.dl_url: "https://syncandshare.lrz.de/dl/fiD9HnHYkb2V8kNbVQYv242m/rv32gc.tar.gz"
flags: {}

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

### Usage

The following examples demonstrate the `tvm_profile` of the TVM and MicroTVM platform.

#### A) Command Line Interface

First define a simple benchmark of a single model/backend/target combination:

In [12]:
!python -m mlonmcu.cli.main flow run resnet -b tvmllvm -t tvm_cpu

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-58]  Processing stage LOAD


INFO - [session-58]  Processing stage BUILD


INFO - [session-58]  Processing stage RUN


INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-58] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework  Backend Platform   Target  Runtime [s] Features                                             Config Postprocesses Comment
0       58    0  resnet   tflite       tvm  tvmllvm      tvm  tvm_cpu      0.00351       []  {'resnet.output_shapes': {'Identity_int8': [1,...            []       -


To enable TVM's profiling feature just just add `-f tvm_profile` to the command line:

In [13]:
!python -m mlonmcu.cli.main flow run resnet -b tvmllvm -t tvm_cpu -f tvm_profile

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-59]  Processing stage LOAD
INFO - [session-59]  Processing stage BUILD


INFO - [session-59]  Processing stage RUN


INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-59] Done processing runs


INFO - Report:
    Session  Run   Model Frontend Framework  Backend Platform   Target                                                Sub   Runtime [s]       Features                                             Config Postprocesses Comment
0        59    0  resnet   tflite       tvm  tvmllvm      tvm  tvm_cpu                                                NaN  1.309400e-03  [tvm_profile]  {'resnet.output_shapes': {'Identity_int8': [1,...            []       -
1        59    0  resnet   tflite       tvm  tvmllvm      tvm  tvm_cpu  tvmgen_default_fused_cast_subtract_fixed_point...  5.091000e-04  [tvm_profile]  {'resnet.output_shapes': {'Identity_int8': [1,...            []       -
2        59    0  resnet   tflite       tvm  tvmllvm      tvm  tvm_cpu  tvmgen_default_fused_nn_conv2d_add_fixed_point...  4.914000e-04  [tvm_profile]  {'resnet.output_shapes': {'Identity_int8': [1,...            []       -
3        59    0  resnet   tflite       tvm  tvmllvm      tvm  tvm_cpu  tvmgen_default_fu

Since tvm used quite long function-names, this might not be very reaible. As a last step, let's try to improve that using the `filter_cols` postprocess:

In [14]:
!python -m mlonmcu.cli.main flow run resnet -b tvmllvm -t tvm_cpu -f tvm_profile \
        --postprocess filter_cols -c filter_cols.keep="Model,Sub,Runtime [s]"

INFO - Loading environment cache from file


INFO - Successfully initialized cache


INFO - [session-60]  Processing stage LOAD
INFO - [session-60]  Processing stage BUILD


INFO - [session-60]  Processing stage RUN


INFO - [session-60]  Processing stage POSTPROCESS
INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-60] Done processing runs
INFO - Report:
     Model                                                Sub   Runtime [s]
0   resnet                                                NaN  1.902400e-03
1   resnet  tvmgen_default_fused_cast_subtract_fixed_point...  4.690600e-04
2   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point...  4.544100e-04
3   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point...  4.486100e-04
4   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point...  2.316800e-04
5   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point...  1.055400e-04
6   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point...  9.271000e-05
7   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point...  6.112000e-05
8   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point...  3.859000e-05
9   resnet  tvmgen_default_fused_nn_conv2d_add_fixed_point

#### B) Python Scripting

Some imports

In [15]:
from tempfile import TemporaryDirectory
from pathlib import Path
import pandas as pd

from mlonmcu.context.context import MlonMcuContext
from mlonmcu.session.run import RunStage

Benchmark Configuration

In [16]:
FRONTEND = "tflite"
MODEL = "resnet"
BACKEND = "tvmllvm"
PLATFORM = "tvm"
TARGET = "tvm_cpu"
FEATURES = ["tvm_profile"]
CONFIG = {}
POSTPROCESSES = []

Initialize and run a single benchmark

In [17]:
with MlonMcuContext() as context:
    with context.create_session() as session:
        run = session.create_run(config=CONFIG)
        run.add_features_by_name(FEATURES, context=context)
        run.add_frontend_by_name(FRONTEND, context=context)
        run.add_model_by_name(MODEL, context=context)
        run.add_platform_by_name(PLATFORM, context=context)
        run.add_backend_by_name(BACKEND, context=context)
        run.add_target_by_name(TARGET, context=context)
        run.add_postprocesses_by_name(POSTPROCESSES)
        session.process_runs(context=context)
        report = session.get_reports()
assert "Failing" not in report.df.columns
report.df

INFO - Loading environment cache from file


INFO - Successfully initialized cache


INFO - [session-61] Processing all stages


INFO - All runs completed successfuly!


INFO - Postprocessing session report


INFO - [session-61] Done processing runs


,Session,Run,Model,Frontend,Framework,Backend,Platform,Target,Sub,Runtime [s],Features,Config,Postprocesses,Comment
0,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,NaN,0.002109,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
1,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_cast_subtract_fixed_point...,0.000819,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
2,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000800,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
3,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000767,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
4,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000405,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
5,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000185,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
6,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000116,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
7,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000094,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
8,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000046,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-
9,61,0,resnet,tflite,tvm,tvmllvm,tvm,tvm_cpu,tvmgen_default_fused_nn_conv2d_add_fixed_point...,0.000023,[tvm_profile],"{'resnet.output_shapes': {'Identity_int8': [1,...",[],-


After stripping it down to the essential data:

In [18]:
df = report.df
df.drop(["Session", "Run", "Frontend", "Model", "Framework", "Backend", "Platform", "Target", "Config", "Features", "Postprocesses", "Comment"], axis=1, inplace=True)
df.fillna("full", inplace=True)
df.set_index("Sub", inplace=True)
df

,Runtime [s]
Sub,
full,0.002109
tvmgen_default_fused_cast_subtract_fixed_point_multiply_add_nn_conv2d_add_fixed_point_multiply__cc9246e62aa5afb_,0.000819
tvmgen_default_fused_nn_conv2d_add_fixed_point_multiply_per_axis_add_clip_cast_subtract,0.000800
tvmgen_default_fused_nn_conv2d_add_fixed_point_multiply_per_axis_add_clip_subtract_fixed_point__eb606f94f03ebac6_,0.000767
tvmgen_default_fused_nn_conv2d_add_fixed_point_multiply_per_axis_add_clip_cast_subtract_1,0.000405
tvmgen_default_fused_nn_conv2d_add_fixed_point_multiply_per_axis_add_clip_subtract_fixed_point__eb606f94f03ebac6__1,0.000185
tvmgen_default_fused_nn_conv2d_add_fixed_point_multiply_per_axis_add_clip_cast,0.000116
tvmgen_default_fused_nn_conv2d_add_fixed_point_multiply_per_axis_add_clip_cast_subtract_2,0.000094
tvmgen_default_fused_nn_conv2d_add_fixed_point_multiply_per_axis_add_clip_subtract_fixed_point__26c49bbe582da641_,0.000046
